In [ ]:
import os
import pandas as pd
import numpy as np
from pandas import read_csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import ADASYN

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df=pd.read_excel('/content/gdrive/MyDrive/Btech_Project/CombinedDataset/Ecl_Myl_pde.xlsx')
df

,classname,numberOfBugsFoundUntil:,numberOfNonTrivialBugsFoundUntil:,numberOfMajorBugsFoundUntil:,numberOfCriticalBugsFoundUntil:,numberOfHighPriorityBugsFoundUntil:,Nondefective,nonTrivialBugs,majorBugs,criticalBugs,...,numberOfAttributesInherited,numberOfLinesOfCode,numberOfMethods,numberOfMethodsInherited,numberOfPrivateAttributes,numberOfPrivateMethods,numberOfPublicAttributes,numberOfPublicMethods,rfc,wmc
0,org::eclipse::mylyn::internal::trac::core::uti...,6,4,0,0,1,1,0,0,0,...,0,42,6,9,0,1,0,5,18,8
1,org::eclipse::mylyn::internal::sandbox::bridge...,12,6,0,0,10,1,0,0,0,...,0,60,9,0,6,0,0,8,29,14
2,org::eclipse::mylyn::internal::jira::core::mod...,6,3,0,0,5,1,0,0,0,...,0,13,4,19,3,0,0,3,5,4
3,org::eclipse::mylyn::internal::wikitext::texti...,3,3,0,0,0,1,0,0,0,...,0,12,4,22,0,0,0,1,6,3
4,org::eclipse::mylyn::internal::java::ui::searc...,3,0,0,0,2,1,0,0,0,...,8,12,4,47,0,0,0,2,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4351,org::eclipse::jdt::internal::compiler::codegen...,10,9,2,2,2,1,0,0,0,...,0,3,2,8,0,0,2,2,2,1
4352,org::eclipse::jdt::internal::compiler::ast::Ja...,23,21,3,1,1,1,0,0,0,...,386,48,6,95,0,1,3,3,27,14
4353,org::eclipse::jdt::internal::core::search::mat...,21,18,1,0,0,0,0,0,0,...,52,306,18,27,2,0,6,6,94,88
4354,org::eclipse::jdt::core::dom::RecoveredVariabl...,4,2,0,0,0,1,0,0,0,...,6,87,20,30,2,0,0,19,47,29


In [ ]:
print("defective ",len(df[df['Nondefective'] ==0]))

defective  660


In [ ]:
unlabelled_df = df.loc[(df['Nondefective']==0)&(df[' highPriorityBugs ']==0)&(df[' criticalBugs ']==0)&(df[' majorBugs ']==0)&(df[' nonTrivialBugs ']==0)]

In [ ]:
print(unlabelled_df.shape)

(405, 43)


In [ ]:
labelled_df = df.loc[~((df['Nondefective']==0)&(df[' highPriorityBugs ']==0)&(df[' criticalBugs ']==0)&(df[' majorBugs ']==0)&(df[' nonTrivialBugs ']==0))]

In [ ]:
print(labelled_df.shape)

(3951, 43)


In [ ]:
df.columns

Index(['classname ', ' numberOfBugsFoundUntil: ',
       ' numberOfNonTrivialBugsFoundUntil: ', ' numberOfMajorBugsFoundUntil: ',
       ' numberOfCriticalBugsFoundUntil: ',
       ' numberOfHighPriorityBugsFoundUntil: ', 'Nondefective',
       ' nonTrivialBugs ', ' majorBugs ', ' criticalBugs ',
       ' highPriorityBugs ', ' numberOfVersionsUntil: ',
       ' numberOfFixesUntil: ', ' numberOfRefactoringsUntil: ',
       ' numberOfAuthorsUntil: ', ' linesAddedUntil: ',
       ' maxLinesAddedUntil: ', ' avgLinesAddedUntil: ',
       ' linesRemovedUntil: ', ' maxLinesRemovedUntil: ',
       ' avgLinesRemovedUntil: ', ' codeChurnUntil: ', ' maxCodeChurnUntil: ',
       ' avgCodeChurnUntil: ', ' ageWithRespectTo: ',
       ' weightedAgeWithRespectTo: ', ' cbo ', ' dit ', ' fanIn ', ' fanOut ',
       ' lcom ', ' noc ', ' numberOfAttributes ',
       ' numberOfAttributesInherited ', ' numberOfLinesOfCode ',
       ' numberOfMethods ', ' numberOfMethodsInherited ',
       ' numberOfPrivateA

# Generating **X,y**

In [ ]:
data =labelled_df.values
idx_OUT_columns = [10,9,8,7,6]
idx_IN_columns = [i for i in range(np.shape(data)[1]) if i not in idx_OUT_columns]
x = data[:,idx_IN_columns]
x = x[:,1:]
y = data[:,idx_OUT_columns]
y=y.astype('int')
print("x : ",x)
print("Y : ",y)

x :  [[6 4 0 ... 5 18 8]
 [12 6 0 ... 8 29 14]
 [6 3 0 ... 3 5 4]
 ...
 [10 9 2 ... 2 2 1]
 [23 21 3 ... 3 27 14]
 [4 2 0 ... 19 47 29]]
Y :  [[0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 ...
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]]


In [ ]:
data =unlabelled_df.values
idx_OUT_columns = [10,9,8,7,6]
idx_IN_columns = [i for i in range(np.shape(data)[1]) if i not in idx_OUT_columns]
X_unlabelled = data[:,idx_IN_columns]
X_unlabelled = X_unlabelled[:,1:]

In [ ]:
oversample = ADASYN(n_neighbors=1)
X, Y = oversample.fit_resample(x,y)

# **CLASSESS**

In [ ]:
def counting(y):
  count=[0,0,0,0,0]
  for i in range(len(y)):
    k=0
    for j in range(5):
      if(y[i][j]==1):
        count[j]+=1
  print("no of nonTrivialBugs ",count[3])
  print("no of majorBugs ",count[2])
  print("no of criticalBugs ",count[1])
  print("no of highPriorityBugs ",count[0])
  print("no of Nondefective modules ",count[4])
  return count;

In [ ]:
counting(Y)

no of nonTrivialBugs  3651
no of majorBugs  3717
no of criticalBugs  3690
no of highPriorityBugs  3709
no of Nondefective modules  3696


[3709, 3690, 3717, 3651, 3696]

In [ ]:
Y=np.argmax(Y,axis=1)

In [ ]:
def Function(clf):
  clf.fit(X, Y)
  y_hat_train = clf.predict(X)
  train_f1 = f1_score(Y, y_hat_train,average='micro')
  print(f"Train f1: {train_f1}")
  df=pd.read_excel('/content/gdrive/MyDrive/Btech_Project/CombinedDataset/equinox.xlsx')
  labelled_df = df.loc[~((df['Nondefective']==0)&(df[' highPriorityBugs ']==0)&(df[' criticalBugs ']==0)&(df[' majorBugs ']==0)&(df[' nonTrivialBugs ']==0))]
  data =labelled_df.values
  idx_OUT_columns = [10,9,8,7,6]
  idx_IN_columns = [i for i in range(np.shape(data)[1]) if i not in idx_OUT_columns]
  x_test = data[:,idx_IN_columns]
  x_test = x_test[:,1:]
  y_test = data[:,idx_OUT_columns]
  y_test=y_test.astype('int')
  y_test=np.argmax(y_test,axis=1)
  yhat_test= clf.predict(x_test)
  yhat_pred_probs = clf.predict_proba(x_test)
  print("test f1score : ",f1_score(y_test, yhat_test,average='micro'))
  CM=confusion_matrix(y_test, yhat_test,labels=[0,1,2,3,4])

  print(CM)

  Risk_HP=((0.1*CM[0][1])+(0.2*CM[0][2])+(0.3*CM[0][3])+(0.4*CM[0][4]))/(CM[0][0]+CM[0][1]+CM[0][2]+CM[0][3]+CM[0][4])
  Risk_C=((0.1*CM[1][2])+(0.2*CM[1][3])+(0.3*CM[1][4]))/(CM[1][0]+CM[1][1]+CM[1][2]+CM[1][3]+CM[1][4])
  Risk_M=((0.1*CM[2][3])+(0.2*CM[2][4]))/(CM[2][0]+CM[2][1]+CM[2][2]+CM[2][3]+CM[2][4])
  Risk_NT=(0.1*CM[3][4])/(CM[3][0]+CM[3][1]+CM[3][2]+CM[3][3]+CM[4][4])
  print("Risk Factor for High Priority",Risk_HP)
  print("Risk Factor for Critical",Risk_C)
  print("Risk Factor for Major",Risk_M)
  print('PTN:',CM[4][4],'/',len(x_test),'=',CM[4][4]/len(x_test))
  total_loc=0
  tn_loc=0
  for i in range(len(x_test)):
    total_loc=total_loc+x_test[i][28]
    if((yhat_test[i]==4)and(y_test[i]==4)):
      tn_loc=tn_loc+x_test[i][28]
  print('Saved Budget:',tn_loc)
  print('PSB:',tn_loc/total_loc)
  print('PNTN:',(1-(CM[4][4]/len(x_test))))
  print('Remaining Servie Time:',total_loc-tn_loc)
  print('PRST:',(1-(tn_loc/total_loc)))

In [ ]:
Function(LogisticRegression());

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Train f1: 0.4986188593403022
test f1score :  0.7326732673267327
[[  0   0   0   0   0]
 [  0   0   0   0   1]
 [  0   1   0   1   1]
 [  1   0   1   0   1]
 [  7   5  34   1 148]]
Risk Factor for High Priority nan
Risk Factor for Critical 0.3
Risk Factor for Major 0.10000000000000002
PTN: 148 / 202 = 0.7326732673267327
Saved Budget: 3386
PSB: 0.25564363910909776
PNTN: 0.26732673267326734
Remaining Servie Time: 9859
PRST: 0.7443563608909023


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
Function(DecisionTreeClassifier())

Train f1: 1.0
test f1score :  0.9405940594059405
[[  0   0   0   0   0]
 [  0   0   0   0   1]
 [  0   0   1   0   2]
 [  0   0   0   0   3]
 [  0   2   4   0 189]]
Risk Factor for High Priority nan
Risk Factor for Critical 0.3
Risk Factor for Major 0.13333333333333333
PTN: 189 / 202 = 0.9356435643564357
Saved Budget: 9908
PSB: 0.7480558701396753
PNTN: 0.0643564356435643
Remaining Servie Time: 3337
PRST: 0.25194412986032466


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
